In [2]:
import numpy as np
import pandas as pd
from dash import html, callback
from dash.dependencies import Input, Output
import dash
import dash_bootstrap_components as dbc
import pandas as pd
from dash import Input, Output, callback, dcc, html
from pathlib import Path




In [8]:
df = pd.read_csv("/home/mukullight/Music/hackealt/eval/data/processed/combined_benchmarks.csv")

In [43]:
import dash
from dash import dcc, html, Output, Input
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# === Extended Data with all years ===
capabilities_data = {
    "Code Generation": {
        "scores": [36, 37.9, 39.9, 42, 44.2, 46.5, 57.5],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "coding"
    },
    "Mathematical Reasoning": {
        "scores": [67, 70.6, 74.3, 78.2, 84.5, 87.9, 91.2],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "mathematics"
    },
    "Commonsense Reasoning": {
        "scores": [41.1, 53, 64.9, 77.2, 81.8, 91.2, 93.9],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "reasoning"
    },
    "Creative Writing": {
        "scores": [41.1, 43.3, 45.5, 47.9, 50.4, 53.1, 100],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "language"
    },
    "Reading Comprehension": {
        "scores": [22.4, 42.7, 63, 68, 86, 88, 90.6],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "language"
    },
    "General Knowledge": {
        "scores": [54.1, 57, 60, 70, 86.4, 88.1, 90.7],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "knowledge"
    },
    "Game Playing": {
        "scores": [25.2, 26.6, 28, 29.4, 31, 32.6, 43.6],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "games"
    },
    "Physical Intuition": {
        "scores": [70.5, 76.1, 81.8, 83.2, 84.9, 88.7, 91.4],
        "years": [2019, 2020, 2021, 2022, 2023, 2024, 2025],
        "category": "reasoning"
    }
}

category_colors = {
    'coding': '#3b82f6',
    'mathematics': '#8b5cf6',
    'reasoning': '#10b981',
    'knowledge': '#f59e0b',
    'language': '#ec4899',
    'games': '#14b8a6'
}

def calculate_stats(selected_caps):
    if not selected_caps:
        return 0, 0, 0, 0
    improvements = []
    growth_rates = []
    current_scores = []
    for cap in selected_caps:
        data = capabilities_data[cap]
        scores = data["scores"]
        years = data["years"]
        improvement = scores[-1] - scores[0]
        improvements.append(improvement)
        if len(years) > 1:
            total_growth = (scores[-1] - scores[0]) / scores[0] if scores[0] != 0 else 0
            avg_annual = (total_growth / (len(years) - 1)) * 100
            growth_rates.append(avg_annual)
        current_scores.append(scores[-1])
    avg_improvement = sum(improvements) / len(improvements)
    avg_growth = sum(growth_rates) / len(growth_rates) if growth_rates else 0
    max_score = max(current_scores) if current_scores else 0
    min_score = min(current_scores) if current_scores else 0
    return avg_improvement, avg_growth, max_score, min_score

def build_figure(selected_caps):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("📈 Timeline Evolution", "📊 2019 vs 2025 Improvement", 
                       "⚡ Annual Growth Rate", ""),
        specs=[[{"colspan": 2}, None],
               [{"type": "bar"}, {"type": "bar"}]],
        row_heights=[0.6, 0.4],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    # Timeline chart (no legend)
    for name, data in capabilities_data.items():
        if name in selected_caps:
            color = category_colors.get(data["category"], '#ffffff')
            fig.add_trace(
                go.Scatter(
                    x=data["years"],
                    y=data["scores"],
                    mode="lines+markers",
                    name=name,
                    line=dict(color=color, width=3),
                    marker=dict(size=10, line=dict(width=2, color='white')),
                    hovertemplate="<b>%{fullData.name}</b><br>" +
                                 "Year: %{x}<br>" +
                                 "Score: %{y:.1f}%<br>" +
                                 "<extra></extra>",
                    showlegend=False
                ),
                row=1, col=1
            )
    # Bar Chart 1: Improvement (2019 vs 2025)
    improvements = []
    names_for_bar = []
    colors_for_bar = []
    for name in selected_caps:
        data = capabilities_data[name]
        improvement = data["scores"][-1] - data["scores"][0]
        improvements.append(improvement)
        names_for_bar.append(name[:20])
        colors_for_bar.append(category_colors.get(data["category"], '#ffffff'))
    fig.add_trace(
        go.Bar(
            y=names_for_bar,
            x=improvements,
            orientation='h',
            marker=dict(color=colors_for_bar, line=dict(color='white', width=1.5)),
            text=[f"+{imp:.1f}%" for imp in improvements],
            textposition='outside',
            hovertemplate="<b>%{y}</b><br>Improvement: +%{x:.1f}%<extra></extra>",
            showlegend=False
        ),
        row=2, col=1
    )
    # Bar Chart 2: Annual Growth Rate
    growth_rates = []
    names_for_growth = []
    colors_for_growth = []
    for name in selected_caps:
        data = capabilities_data[name]
        scores = data["scores"]
        years = data["years"]
        if len(years) > 1:
            total_growth = (scores[-1] - scores[0]) / scores[0] if scores[0] != 0 else 0
            avg_annual = (total_growth / (len(years) - 1)) * 100
            growth_rates.append(avg_annual)
            names_for_growth.append(name[:20])
            colors_for_growth.append(category_colors.get(data["category"], '#ffffff'))
    if growth_rates:
        sorted_data = sorted(zip(growth_rates, names_for_growth, colors_for_growth), reverse=True)
        growth_rates, names_for_growth, colors_for_growth = zip(*sorted_data)
    fig.add_trace(
        go.Bar(
            y=list(names_for_growth),
            x=list(growth_rates),
            orientation='h',
            marker=dict(color=list(colors_for_growth), line=dict(color='white', width=1.5)),
            text=[f"{rate:.1f}%" for rate in growth_rates],
            textposition='outside',
            hovertemplate="<b>%{y}</b><br>Annual Growth: %{x:.1f}%<extra></extra>",
            showlegend=False
        ),
        row=2, col=2
    )
    fig.update_xaxes(title_text="Year", row=1, col=1, gridcolor='rgba(255,255,255,0.1)')
    fig.update_yaxes(title_text="Performance Score (%)", row=1, col=1, gridcolor='rgba(255,255,255,0.1)')
    fig.update_xaxes(title_text="Improvement (%)", row=2, col=1, gridcolor='rgba(255,255,255,0.1)')
    fig.update_xaxes(title_text="Annual Growth (%)", row=2, col=2, gridcolor='rgba(255,255,255,0.1)')
    fig.update_layout(
        template="plotly_dark",
        height=900,
        showlegend=False,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(0,0,0,0)"
        ),
        title={
            'text': "🚀 AI Model Capability Evolution Dashboard (2019–2025)",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 24, 'color': 'white'}
        },
        paper_bgcolor='#0a0a15',
        plot_bgcolor='#12121e',
        font=dict(color='white', size=12)
    )
    return fig

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("🎯 AI Capability Timeline Dashboard", 
                    className="text-center text-white mb-2"),
            html.P("Interactive visualization of AI model progress across different capabilities",
                   className="text-center text-secondary mb-4"),
            dbc.Card([
                dbc.CardHeader(html.H5("✓ Select Capabilities", className="mb-0")),
                dbc.CardBody([
                    dcc.Checklist(
                        id="capability-checklist",
                        options=[{"label": cap, "value": cap} for cap in capabilities_data.keys()],
                        value=["Code Generation", "Mathematical Reasoning", "Commonsense Reasoning"],
                        inline=False,
                        inputStyle={"marginRight": "10px"},
                        labelStyle={
                            "display": "block", 
                            "color": "white", 
                            "padding": "8px",
                            "fontSize": "14px",
                            "cursor": "pointer",
                            "borderRadius": "4px",
                            "marginBottom": "5px"
                        },
                        style={"padding": "10px"}
                    )
                ])
            ], color="dark", inverse=True, className="mb-3"),
            dbc.Card([
                dbc.CardHeader(html.H5("📊 Statistics", className="mb-0")),
                dbc.CardBody(html.Div(id="stats-display"))
            ], color="dark", inverse=True, className="mb-3"),
            dbc.Card([
                dbc.CardHeader(html.H6("🎨 Categories")),
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col(html.Div("●", style={"color": color, "fontSize": "18px"}), width="auto"),
                        dbc.Col(html.Div(cat.title(), style={"color": "white"}))
                    ], align="center", className="mb-1")
                    for cat, color in category_colors.items()
                ])
            ], color="dark", inverse=True)
        ], width=3),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(
                        id="capability-graph", 
                        figure=build_figure(["Code Generation", "Mathematical Reasoning", "Commonsense Reasoning"]),
                        config={'displayModeBar': True, 'displaylogo': False}
                    )
                ])
            ], color="dark", inverse=True)
        ], width=9)
    ]),
    dbc.Row([
        dbc.Col([
            html.P("💡 Instructions: Check/uncheck boxes to compare different AI capabilities • "
                   "Hover over charts for detailed information • All data normalized to 0-100% scale",
                   className="text-center text-secondary mt-3")
        ])
    ])
], fluid=True, style={"backgroundColor": "#0a0a15", "minHeight": "100vh"})

@app.callback(
    [Output("capability-graph", "figure"),
     Output("stats-display", "children")],
    Input("capability-checklist", "value")
)
def update_dashboard(selected_caps):
    fig = build_figure(selected_caps)
    avg_imp, avg_growth, max_score, min_score = calculate_stats(selected_caps)
    stats_content = [
        html.Div([
            html.Strong("Active Capabilities: ", style={"color": "#3b82f6"}),
            html.Span(f"{len(selected_caps)}/{len(capabilities_data)}")
        ], style={"marginBottom": "10px", "fontSize": "14px"}),
        html.Div([
            html.Strong("Avg Improvement: ", style={"color": "#10b981"}),
            html.Span(f"+{avg_imp:.1f}%")
        ], style={"marginBottom": "10px", "fontSize": "14px"}),
        html.Div([
            html.Strong("Avg Annual Growth: ", style={"color": "#f59e0b"}),
            html.Span(f"{avg_growth:.1f}%")
        ], style={"marginBottom": "10px", "fontSize": "14px"}),
        html.Div([
            html.Strong("Score Range: ", style={"color": "#ec4899"}),
            html.Span(f"{min_score:.1f}% - {max_score:.1f}%")
        ], style={"fontSize": "14px"})
    ]
    return fig, stats_content

if __name__ == "__main__":
    app.run_server(debug=True)



🚀 AI MODEL CAPABILITY DASHBOARD - DASH APPLICATION

✅ Starting server...
📊 Features:
   • Interactive timeline with 8 AI capabilities
   • Bar charts for improvement and growth analysis
   • Real-time statistics updates
   • Checkbox selection interface

🌐 Open your browser and navigate to: http://127.0.0.1:8050/



In [ ]:
import plotly.express as px
import plotly.graph_objects as go


def graph_map(df: pd.DataFrame) -> go.Figure:
    """Create a scatter plot of models by country and score."""
    fig = px.scatter_geo(
        df,
        locations="country",
        locationmode="country names",
        color="score",
        hover_name="model",
        size="score",
        projection="natural earth",
    )
    return fig


In [9]:
df.columns

Index(['model', 'benchmark', 'date', 'org', 'country',
       'training_compute_flops', 'score', 'capability'],
      dtype='object')

In [48]:
import dash
from dash import dcc, html, Output, Input
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# === Extended Data with all years ===
capabilities_data = data
category_colors = {
    'coding': '#3b82f6',
    'mathematics': '#8b5cf6',
    'reasoning': '#10b981',
    'knowledge': '#f59e0b',
    'language': '#ec4899',
    'games': '#14b8a6'
}

# === Calculate statistics ===
def calculate_stats(selected_caps):
    if not selected_caps:
        return 0, 0, 0, 0
    
    improvements = []
    growth_rates = []
    current_scores = []
    
    for cap in selected_caps:
        data = capabilities_data[cap]
        scores = data["scores"]
        years = data["years"]
        
        # Improvement
        improvement = scores[-1] - scores[0]
        improvements.append(improvement)
        
        # Annual growth rate
        if len(years) > 1:
            total_growth = (scores[-1] - scores[0]) / scores[0] if scores[0] != 0 else 0
            avg_annual = (total_growth / (len(years) - 1)) * 100
            growth_rates.append(avg_annual)
        
        # Current score
        current_scores.append(scores[-1])
    
    avg_improvement = sum(improvements) / len(improvements)
    avg_growth = sum(growth_rates) / len(growth_rates) if growth_rates else 0
    max_score = max(current_scores) if current_scores else 0
    min_score = min(current_scores) if current_scores else 0
    
    return avg_improvement, avg_growth, max_score, min_score

# === Build comprehensive figure with subplots ===
def build_figure(selected_caps):
    # Create subplots: 1 main chart + 2 bar charts
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("📈 Timeline Evolution", "📊 2019 vs 2025 Improvement", 
                       "⚡ Annual Growth Rate", ""),
        specs=[[{"colspan": 2}, None],
               [{"type": "bar"}, {"type": "bar"}]],
        row_heights=[0.6, 0.4],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # === Main Timeline Chart ===
    for name, data in capabilities_data.items():
        if name in selected_caps:
            color = category_colors.get(data["category"], '#ffffff')
            fig.add_trace(
                go.Scatter(
                    x=data["years"],
                    y=data["scores"],
                    mode="lines+markers",
                    name=name,
                    line=dict(color=color, width=3),
                    marker=dict(size=10, line=dict(width=2, color='white')),
                    hovertemplate="<b>%{fullData.name}</b><br>" +
                                 "Year: %{x}<br>" +
                                 "Score: %{y:.1f}%<br>" +
                                 "<extra></extra>"
                ),
                row=1, col=1
            )
    
    # === Bar Chart 1: Improvement (2019 vs 2025) ===
    improvements = []
    names_for_bar = []
    colors_for_bar = []
    
    for name in selected_caps:
        data = capabilities_data[name]
        improvement = data["scores"][-1] - data["scores"][0]
        improvements.append(improvement)
        names_for_bar.append(name[:20])  # Truncate long names
        colors_for_bar.append(category_colors.get(data["category"], '#ffffff'))
    
    fig.add_trace(
        go.Bar(
            y=names_for_bar,
            x=improvements,
            orientation='h',
            marker=dict(color=colors_for_bar, line=dict(color='white', width=1.5)),
            text=[f"+{imp:.1f}%" for imp in improvements],
            textposition='outside',
            hovertemplate="<b>%{y}</b><br>Improvement: +%{x:.1f}%<extra></extra>",
            showlegend=False
        ),
        row=2, col=1
    )
    
    # === Bar Chart 2: Annual Growth Rate ===
    growth_rates = []
    names_for_growth = []
    colors_for_growth = []
    
    for name in selected_caps:
        data = capabilities_data[name]
        scores = data["scores"]
        years = data["years"]
        
        if len(years) > 1:
            total_growth = (scores[-1] - scores[0]) / scores[0] if scores[0] != 0 else 0
            avg_annual = (total_growth / (len(years) - 1)) * 100
            growth_rates.append(avg_annual)
            names_for_growth.append(name[:20])
            colors_for_growth.append(category_colors.get(data["category"], '#ffffff'))
    
    # Sort by growth rate
    if growth_rates:
        sorted_data = sorted(zip(growth_rates, names_for_growth, colors_for_growth), reverse=True)
        growth_rates, names_for_growth, colors_for_growth = zip(*sorted_data)
    
    fig.add_trace(
        go.Bar(
            y=list(names_for_growth),
            x=list(growth_rates),
            orientation='h',
            marker=dict(color=list(colors_for_growth), line=dict(color='white', width=1.5)),
            text=[f"{rate:.1f}%" for rate in growth_rates],
            textposition='outside',
            hovertemplate="<b>%{y}</b><br>Annual Growth: %{x:.1f}%<extra></extra>",
            showlegend=False
        ),
        row=2, col=2
    )
    
    # === Update Layout ===
    fig.update_xaxes(title_text="Year", row=1, col=1, gridcolor='rgba(255,255,255,0.1)')
    fig.update_yaxes(title_text="Performance Score (%)", row=1, col=1, gridcolor='rgba(255,255,255,0.1)')
    
    fig.update_xaxes(title_text="Improvement (%)", row=2, col=1, gridcolor='rgba(255,255,255,0.1)')
    fig.update_xaxes(title_text="Annual Growth (%)", row=2, col=2, gridcolor='rgba(255,255,255,0.1)')
    
    fig.update_layout(
        template="plotly_dark",
        height=900,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(0,0,0,0)"
        ),
        title={
            'text': "🚀 AI Model Capability Evolution Dashboard (2019–2025)",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 24, 'color': 'white'}
        },
        paper_bgcolor='#0a0a15',
        plot_bgcolor='#12121e',
        font=dict(color='white', size=12)
    )
    
    return fig

# === Dash app ===
app = dash.Dash(__name__)

app.layout = html.Div([
    # Header
    html.Div([
        html.H1("🎯 AI Capability Timeline Dashboard", 
               style={"textAlign": "center", "color": "white", "marginBottom": "10px"}),
        html.P("Interactive visualization of AI model progress across different capabilities",
              style={"textAlign": "center", "color": "lightgray", "fontSize": "16px"})
    ], style={"padding": "20px", "backgroundColor": "#1a1a2e"}),
    
    # Main content
    html.Div([
        # Left sidebar with checkboxes and stats
        html.Div([
            # Checkbox section
            html.Div([
                html.H3("✓ Select Capabilities", 
                       style={"color": "white", "textAlign": "center", 
                             "backgroundColor": "#2a2a4e", "padding": "10px",
                             "borderRadius": "8px", "marginBottom": "15px"}),
                dcc.Checklist(
                    id="capability-checklist",
                    options=[{"label": cap, "value": cap} for cap in capabilities_data.keys()],
                    value=["Code Generation", "Mathematical Reasoning", "Commonsense Reasoning"],
                    inline=False,
                    inputStyle={"marginRight": "10px"},
                    labelStyle={
                        "display": "block", 
                        "color": "white", 
                        "padding": "8px",
                        "fontSize": "14px",
                        "cursor": "pointer",
                        "borderRadius": "4px",
                        "marginBottom": "5px"
                    },
                    style={"padding": "10px"}
                )
            ], style={
                "backgroundColor": "#1a1a2e", 
                "padding": "15px", 
                "borderRadius": "10px",
                "border": "2px solid white",
                "marginBottom": "20px"
            }),
            
            # Statistics section
            html.Div([
                html.H3("📊 Statistics", 
                       style={"color": "white", "textAlign": "center",
                             "backgroundColor": "#2a2a4e", "padding": "10px",
                             "borderRadius": "8px", "marginBottom": "15px"}),
                html.Div(id="stats-display", style={"padding": "10px", "color": "white"})
            ], style={
                "backgroundColor": "#1a1a2e",
                "padding": "15px",
                "borderRadius": "10px",
                "border": "2px solid white"
            }),
            
            # Category legend
            html.Div([
                html.H4("🎨 Categories", style={"color": "white", "marginBottom": "10px"}),
                html.Div([
                    html.Div([
                        html.Span("●", style={"color": color, "fontSize": "20px", "marginRight": "8px"}),
                        html.Span(cat.title(), style={"color": "white"})
                    ], style={"marginBottom": "5px"})
                    for cat, color in category_colors.items()
                ])
            ], style={
                "backgroundColor": "#1a1a2e",
                "padding": "15px",
                "borderRadius": "10px",
                "border": "1.5px solid white",
                "marginTop": "20px"
            })
            
        ], style={
            "width": "20%", 
            "display": "inline-block", 
            "verticalAlign": "top", 
            "padding": "20px"
        }),
        
        # Right side with graph
        html.Div([
            dcc.Graph(
                id="capability-graph", 
                figure=build_figure(["Code Generation", "Mathematical Reasoning", "Commonsense Reasoning"]),
                config={'displayModeBar': True, 'displaylogo': False}
            )
        ], style={
            "width": "78%", 
            "display": "inline-block", 
            "padding": "20px"
        }),
    ]),
    
    # Footer
    html.Div([
        html.P("💡 Instructions: Check/uncheck boxes to compare different AI capabilities • "
               "Hover over charts for detailed information • All data normalized to 0-100% scale",
               style={"textAlign": "center", "color": "lightgray", "margin": "0"})
    ], style={"padding": "15px", "backgroundColor": "#1a1a2e"})
    
], style={"backgroundColor": "#0a0a15", "fontFamily": "Arial", "minHeight": "100vh"})

# === Callbacks ===
@app.callback(
    [Output("capability-graph", "figure"),
     Output("stats-display", "children")],
    Input("capability-checklist", "value")
)
def update_dashboard(selected_caps):
    # Update figure
    fig = build_figure(selected_caps)
    
    # Calculate and format statistics
    avg_imp, avg_growth, max_score, min_score = calculate_stats(selected_caps)
    
    stats_content = [
        html.Div([
            html.Strong("Active Capabilities: ", style={"color": "#3b82f6"}),
            html.Span(f"{len(selected_caps)}/{len(capabilities_data)}")
        ], style={"marginBottom": "10px", "fontSize": "14px"}),
        
        html.Div([
            html.Strong("Avg Improvement: ", style={"color": "#10b981"}),
            html.Span(f"+{avg_imp:.1f}%")
        ], style={"marginBottom": "10px", "fontSize": "14px"}),
        
        html.Div([
            html.Strong("Avg Annual Growth: ", style={"color": "#f59e0b"}),
            html.Span(f"{avg_growth:.1f}%")
        ], style={"marginBottom": "10px", "fontSize": "14px"}),
        
        html.Div([
            html.Strong("Score Range: ", style={"color": "#ec4899"}),
            html.Span(f"{min_score:.1f}% - {max_score:.1f}%")
        ], style={"fontSize": "14px"})
    ]
    
    return fig, stats_content

if __name__ == "__main__":
    print("\n" + "="*70)
    print("🚀 AI MODEL CAPABILITY DASHBOARD - DASH APPLICATION")
    print("="*70)
    print("\n✅ Starting server...")
    print("📊 Features:")
    print("   • Interactive timeline with 8 AI capabilities")
    print("   • Bar charts for improvement and growth analysis")
    print("   • Real-time statistics updates")
    print("   • Checkbox selection interface")
    print("\n🌐 Open your browser and navigate to: http://127.0.0.1:8050/")
    print("="*70 + "\n")
    
    app.run_server(debug=True)

KeyError: 'Code Generation'